In [1]:
!pip install google-generativeai pymongo python-dotenv scikit-learn pillow ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.5 MB/s eta 0:00:00


In [2]:
import os
import io
import requests
import google.generativeai as genai
import numpy as np
from pymongo import MongoClient
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display
import ipywidgets as widgets

# Set your credentials
os.environ["GEMINI_API_KEY"] = "your gemini key here"
os.environ["MONGO_URI"] = "your_mongodb_connection_uri_here"

# Setup
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
client = MongoClient(os.environ["MONGO_URI"])
db = client["RewindAndRevive"]
collection = db["products"]

conversation_history = []
vectorizer = None
tfidf_matrix = None
products_data = []

def load_data_from_mongo():
    global products_data, vectorizer, tfidf_matrix
    products = list(collection.find())
    if not products:
        raise Exception("No products found in MongoDB.")
    products_data = products
    texts = [
        ' '.join([str(doc.get(field, '')) for field in ['name', 'description', 'category', 'color', 'material', 'type']])
        for doc in products
    ]
    vectorizer = TfidfVectorizer(max_features=300)
    tfidf_matrix = vectorizer.fit_transform(texts)
    print(f"✅ Loaded {len(products)} products.")

def chat_with_bot(user_query, image_bytes=None):
    global conversation_history
    query_vector = vectorizer.transform([user_query])
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-20:][::-1]

    keyword_to_type = {
        "pants": "bottom", "jeans": "bottom", "shorts": "bottom", "skirt": "bottom",
        "shirt": "top", "t-shirt": "top", "blouse": "top", "top": "top",
    }
    detected_type = next((v for k, v in keyword_to_type.items() if k in user_query.lower()), None)

    if image_bytes and not detected_type:
        try:
            model = genai.GenerativeModel("gemini-2.0-flash")
            response = model.generate_content([
                {"mime_type": "image/jpeg", "data": image_bytes},
                {"text": "Is this a top or a bottom? Reply only 'top' or 'bottom'."}
            ])
            if "top" in response.text.lower(): detected_type = "top"
            elif "bottom" in response.text.lower(): detected_type = "bottom"
        except Exception as e:
            print("❌ Error analyzing image:", e)

    recommended_type = "bottom" if detected_type == "top" else "top" if detected_type == "bottom" else None

    def is_type(p, target):
        combined = f"{p.get('name','')} {p.get('description','')} {p.get('type','')}".lower()
        return any(k in combined for k in (["jeans", "skirt", "pants"] if target == "bottom" else ["shirt", "top", "blouse"]))

    candidates = [products_data[i] for i in top_indices if not recommended_type or is_type(products_data[i], recommended_type)]
    recommended = candidates[:3] if candidates else [products_data[i] for i in top_indices[:3]]

    try:
        chat = genai.GenerativeModel("gemini-2.0-flash").start_chat(history=conversation_history)
        prompt = f"{user_query}. Suggest matching items." if detected_type else user_query
        if image_bytes:
            response = chat.send_message([{"mime_type": "image/jpeg", "data": image_bytes}, {"text": prompt}])
        else:
            response = chat.send_message(prompt)
        bot_reply = response.text.strip().split('\n')[0] + '.'
    except Exception as e:
        bot_reply = "Sorry, I had trouble generating a suggestion."
        print("❌ Gemini error:", e)

    conversation_history.extend([
        {"role": "user", "parts": [{"text": user_query}]},
        {"role": "model", "parts": [{"text": bot_reply}]}
    ])

    def format_product(p):
        return {
            "name": p.get("name"),
            "description": p.get("description"),
            "price": p.get("price"),
            "color": p.get("color"),
            "image": p.get("images", [""])[0] if isinstance(p.get("images"), list) else "",
        }

    return {"reply": bot_reply, "products": [format_product(p) for p in recommended]}

def run_bot_ui():
    user_input = widgets.Text(description='Query:', layout=widgets.Layout(width='600px'))
    file_upload = widgets.FileUpload(accept='image/*', multiple=False)
    output = widgets.Output()

    def on_submit(_):
        with output:
            output.clear_output()
            image_bytes = None
            if file_upload.value:
                image_bytes = next(iter(file_upload.value.values()))['content']
            try:
                result = chat_with_bot(user_input.value, image_bytes)
                print("🤖", result["reply"])
                for p in result["products"]:
                    print(f"📦 {p['name']} - {p['description']}")
                    if p["image"]:
                        try:
                            img_data = requests.get(p["image"]).content
                            display(Image.open(io.BytesIO(img_data)))
                        except: print("Image load failed")
            except Exception as e:
                print("Error:", e)

    button = widgets.Button(description="Submit")
    button.on_click(on_submit)
    display(widgets.VBox([user_input, file_upload, button, output]))

# Load data and run UI
load_data_from_mongo()
run_bot_ui()


✅ Loaded 22 products.
